In [1]:
import duckdb
import pandas as pd
import numpy as np


In [2]:
con = duckdb.connect()
con.execute("PRAGMA database_list").fetchall()
con=duckdb.connect('/Users/patriciawong/course/budget_project/budget_project.duckdb')
con.execute('Show Tables').fetchdf()

,name
0,BankA_seed
1,BankB_seed
2,stg_BankA
3,stg_BankB


In [3]:
df_a=con.execute('SELECT distinct merchant_descriptor FROM stg_BankA').fetchdf()

In [4]:
df_b=con.execute('SELECT distinct merchant_descriptor FROM stg_BankB').fetchdf()

In [5]:
df_a.head(3)

,merchant_descriptor
0,ICELANDA 1082416461723
1,TST*CHIPOTLE GRILL
2,United BARCELONA


In [6]:
df_b.head(3)

,merchant_descriptor
0,CHEVRON #1234 ANYTOWN USA
1,COSTCO GAS #0148 ANYTOWN USA
2,DAYCARE 1234 ANYTOWN USA


In [7]:
df_total=pd.concat([df_a, df_b], ignore_index=False)

In [8]:
df_total.sort_values(by=['merchant_descriptor'], ascending=True)

,merchant_descriptor
17,ALDI GROCERY STORE ANYTOWN USA
30,AMAZON MKTPL*NA3S048Z2 Amzn.com/bill WA
15,AMAZON PRIME VIDEO WWW.AMAZON.CO WA
25,AMAZON RETA* NH96X9LV1 WWW.AMAZON.CO WA
9,AMAZON RETA* NH9CH2KO2 WWW.AMAZON.CO WA
...,...
6,United ZURICH
6,WALMART SUPERCENTER ANYTOWN USA
22,WHOLE FOODS MARKET ANYTOWN USA
4,WHOLEFDS CBL 10033 ANYTOWN USA


In [9]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97 entries, 0 to 32
Data columns (total 1 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   merchant_descriptor  97 non-null     object
dtypes: object(1)
memory usage: 1.5+ KB


In [10]:
df_total['merchant_descriptor']=df_total['merchant_descriptor'].astype('string')

In [11]:
df_total['dim_dt']=pd.to_datetime('today')

In [12]:
def merchant_name(row):

    if ('TST' in row['merchant_descriptor']) or ('SQ' in row['merchant_descriptor']):
        return 'Retail - Food Merchant'
    elif 'ALDI' in row['merchant_descriptor']:
        return 'Aldi'
    elif 'AMAZON' in row['merchant_descriptor']:
        return 'Amazon'
    elif 'PAYMENT' in row['merchant_descriptor']:
        return 'Payment'
    elif ('YMCA' in row['merchant_descriptor']) or ('DAYCARE' in row['merchant_descriptor']):
        return 'Childcare'
    elif 'BLUE CROSS' in row['merchant_descriptor']:
        return 'Blue Cross Blue Shield'
    elif 'CHEVRON' in row['merchant_descriptor']:
        return 'Chevron'
    elif 'CHILDREN' in row['merchant_descriptor']:
        return 'Children''s Place'
    elif 'COSTCO' in row['merchant_descriptor']:
        return 'Costco'
    elif 'GEICO' in row['merchant_descriptor']:
        return 'GEICO'
    elif 'HOME DEPOT' in row['merchant_descriptor']:
        return 'The Home Depot'
    elif 'ICELANDA' in row['merchant_descriptor']:
        return 'Icelandic Air'
    elif 'JETBLUE' in row['merchant_descriptor']:
        return 'JetBlue'
    elif 'KROGER' in row['merchant_descriptor']:
        return 'Kroger'
    elif 'NIJIYA' in row['merchant_descriptor']:
        return 'Nijiya'
    elif 'Pets' in row['merchant_descriptor']:
        return 'Retail - Pets'
    elif 'PROGRESSIVE' in row['merchant_descriptor']:
        return 'Progressive'
    elif 'PTA' in row['merchant_descriptor']:
        return 'PTA'
    elif 'SAFEWAY' in row['merchant_descriptor']:
        return 'Safeway'
    elif 'SHELL' in row['merchant_descriptor']:
        return 'Shell Gas'
    elif 'SHOPIFY' in row['merchant_descriptor']:
        return 'Retail - Small Merchant'
    elif 'SPROUTS' in row['merchant_descriptor']:
        return 'Sprouts'
    elif 'SUNCO' in row['merchant_descriptor']:
        return 'Sunco'
    elif 'TARGET' in row['merchant_descriptor']:
        return 'Target'
    elif 'TRADER' in row['merchant_descriptor']:
        return 'Trader Joe''s'
    elif 'UTILITY' in row['merchant_descriptor']:
        return 'Utilities'
    elif 'WALMART' in row['merchant_descriptor']:
        return 'Walmart'     
    elif 'WSJ' in row['merchant_descriptor']:
        return 'Wall Street Journal'     
    elif ('WHOLE FOODS' in row['merchant_descriptor']) or ('WHOLEFDS' in row['merchant_descriptor']):
        return 'Whole Foods'
    elif 'United' in row['merchant_descriptor']:
        return 'United Air'
    else: 
        return 'Needs merchant name'


df_total['merchant_name']=df_total.apply(merchant_name, axis=1)

In [13]:
df_total[df_total['merchant_name']=='Needs merchant name'].head(10)

,merchant_descriptor,dim_dt,merchant_name


In [14]:
df_total[df_total['merchant_name']=='Needs merchant name']['merchant_name'].count()

0

In [30]:
#define category name based off merchant name

def category_name(row):
    if ('Amazon' in row['merchant_name']) or ('Retail' in row['merchant_name']) or ('The Home Depot' in row['merchant_name']) or ('Costco' in row['merchant_name']) or ('Walmart' in row['merchant_name']) or ('Childrens' in row['merchant_name']) or ('Target' in row['merchant_name']) :
        return 'Retail'
    elif ('Chevron' in row['merchant_name']) or ('Shell' in row['merchant_name']) or ('Sunco' in row['merchant_name']):
        return 'Gas'
    elif ('Aldi' in row['merchant_name']) or ('Nijiya' in row['merchant_name']) or ('Kroger' in row['merchant_name']) or ('Safeway' in row['merchant_name']) or ('Sprouts' in row['merchant_name']) or ('Whole Foods' in row['merchant_name']) or ('Trader' in row['merchant_name']):
        return 'Groceries'
    elif  ('Childcare' in row['merchant_name']) or ('Progressive' in row['merchant_name']) or ('GEICO' in row['merchant_name']) or ('Utilities' in row['merchant_name']):
        return 'Bills & Utilities'
    elif 'Blue Cross' in row['merchant_name']:
        return 'Healthcare'
    elif 'Payment' in row['merchant_name']:
        return 'Payment'
    elif ('Wall Street Journal' in row['merchant_name']) or ('PTA' in row['merchant_name']):
        return 'Personal'
    elif ('Icelandic Air' in row['merchant_name']) or ('JetBlue' in row['merchant_name']) or ('United Air' in row['merchant_name']):
        return 'Travel'
    else:
        return 'Needs category name'


df_total['category_name']=df_total.apply(category_name, axis=1)    

In [32]:
df_total.head()

,merchant_descriptor,dim_dt,merchant_name,category_name
0,ICELANDA 1082416461723,2025-08-20 14:04:05.998220,Icelandic Air,Travel
1,TST*CHIPOTLE GRILL,2025-08-20 14:04:05.998220,Retail - Food Merchant,Retail
2,United BARCELONA,2025-08-20 14:04:05.998220,United Air,Travel
3,TST*PANDA EXPRESS,2025-08-20 14:04:05.998220,Retail - Food Merchant,Retail
4,SQ *SONIC DRIVE IN,2025-08-20 14:04:05.998220,Retail - Food Merchant,Retail
